# Transcript A from Lecture, February 2, 2023


In [1]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2023-winter/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# Condition number, residual, and error

In [2]:
A = np.random.random((10,10))

In [3]:
x_exact = np.ones(A.shape[1])
b = A @ x_exact
print('b:', b)

b: [4.5607 5.5798 4.6385 4.5139 3.4155 5.7983 3.9399 3.8312 7.1501 5.6038]


In [4]:
x = npla.solve(A, b)

print('x:', x)


x: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
residual = b - A @ x
error = x_exact - x

print('n:', A.shape[1])
print()
print('relative residual norm:', npla.norm(residual)/npla.norm(b))
print('relative error norm:', npla.norm(error)/npla.norm(x_exact))
print()
print('condition number of A:', npla.cond(A, 2))

n: 10

relative residual norm: 8.395167502665058e-17
relative error norm: 2.885084945932456e-15

condition number of A: 72.41659362588157


# The Hilbert matrix, an ill-conditioned example

In [6]:
def hilbert(n):
    """n-by-n Hilbert matrix, a famous example of an ill-conditioned matrix"""
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            A[i,j] = 1 / (i + j + 1)
    return A

In [7]:
print('hilbert(4):\n', hilbert(4))

hilbert(4):
 [[1.     0.5    0.3333 0.25  ]
 [0.5    0.3333 0.25   0.2   ]
 [0.3333 0.25   0.2    0.1667]
 [0.25   0.2    0.1667 0.1429]]


In [8]:
U, S, Vt = npla.svd(hilbert(4))
print('singular values:', S)

singular values: [1.5002e+00 1.6914e-01 6.7383e-03 9.6702e-05]


In [9]:
A = hilbert(4)
npla.matrix_rank(A)

4

In [10]:
npla.norm(A,2)

1.500214280059243

In [11]:
npla.cond(A,2)

15513.738738928929

In [12]:
A = hilbert(10)
print('hilbert(10):\n', hilbert(10))
print()
U, S, Vt = npla.svd(A)
print('singular values:', S)

hilbert(10):
 [[1.     0.5    0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1   ]
 [0.5    0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909]
 [0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833]
 [0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769]
 [0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714]
 [0.1667 0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667]
 [0.1429 0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625]
 [0.125  0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588]
 [0.1111 0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588 0.0556]
 [0.1    0.0909 0.0833 0.0769 0.0714 0.0667 0.0625 0.0588 0.0556 0.0526]]

singular values: [1.7519e+00 3.4293e-01 3.5742e-02 2.5309e-03 1.2875e-04 4.7297e-06
 1.2290e-07 2.1474e-09 2.2667e-11 1.0932e-13]


In [13]:
x_exact = np.ones(A.shape[1])
b = A @ x_exact
b

array([2.929 , 2.0199, 1.6032, 1.3468, 1.1682, 1.0349, 0.9307, 0.8467,
       0.7773, 0.7188])

In [14]:
x = npla.solve(A, b)

print('x:', x)

x: [1.     1.     1.     1.     1.0001 0.9996 1.0006 0.9994 1.0003 0.9999]


In [15]:
residual = b - A @ x
error = x_exact - x

print('n:', A.shape[1])
print()
print('relative residual norm:', npla.norm(residual)/npla.norm(b))
print('relative error norm:', npla.norm(error)/npla.norm(x_exact))
print()
print('condition number of A:', npla.cond(A, 2))

n: 10

relative residual norm: 1.0000930575135377e-16
relative error norm: 0.00030059447258579624

condition number of A: 16024909625167.58


In [16]:
A = hilbert(20)
x_exact = np.ones(A.shape[1])
b = A @ x_exact
b

array([3.5977, 2.6454, 2.1908, 1.901 , 1.6926, 1.5326, 1.4044, 1.2986,
       1.2093, 1.1327, 1.066 , 1.0074, 0.9553, 0.9087, 0.8666, 0.8286,
       0.7938, 0.762 , 0.7328, 0.7058])

In [17]:
x = npla.solve(A, b)

print('x:', x)

x: [  1.       0.9998   1.0073   0.8847   1.9403  -3.245   11.1563  -6.9711
 -17.4924  49.382  -24.0222 -27.9156  13.1482  38.8323  11.1986 -78.8609
  28.9709  56.3457 -50.3928  14.034 ]


In [18]:
residual = b - A @ x
error = x_exact - x

print('n:', A.shape[1])
print()
print('relative residual norm:', npla.norm(residual)/npla.norm(b))
print('relative error norm:', npla.norm(error)/npla.norm(x_exact))
print()
print('condition number of A:', npla.cond(A, 2))

n: 20

relative residual norm: 5.880725188585429e-16
relative error norm: 30.860877247847647

condition number of A: 1.5315755993591903e+18


In [19]:
U, S, Vt = npla.svd(A)
print('singular values:', S)

singular values: [1.9071e+00 4.8704e-01 7.5596e-02 8.9611e-03 8.6767e-04 7.0334e-05
 4.8305e-06 2.8277e-07 1.4140e-08 6.0361e-10 2.1929e-11 6.7408e-13
 1.7383e-14 3.8010e-16 1.6005e-17 1.2835e-17 9.7655e-18 7.2755e-18
 3.4446e-18 1.2452e-18]


In [20]:
npla.matrix_rank(A)

13

## What about the determinant?

In [21]:
npla.det?

In [22]:
A = np.random.random((5,5))

In [23]:
npla.det(A)

-0.11369445621038594

In [24]:
npla.norm(A)

3.057466182100202

In [25]:
npla.cond(A)

19.477551924215213

<b> The determinant is less useful than norm and condition number, especially as **n** grows.

In [26]:
n = 10
A = 2 * np.eye(n)
print('A:\n', A)

A:
 [[2. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]]


In [27]:
print('n:', n, '\nnorm:', npla.norm(A,2), '\ncondition number:', npla.cond(A), '\ndeterminant:', npla.det(A))

n: 10 
norm: 2.0 
condition number: 1.0 
determinant: 1024.0


In [28]:
n = 100
A = 2 * np.eye(n)

print('n:', n, '\nnorm:', npla.norm(A,2), '\ncondition number:', npla.cond(A), '\ndeterminant:', npla.det(A))

n: 100 
norm: 2.0 
condition number: 1.0 
determinant: 1.2676506002283037e+30
